## 네이버 영화 정보 크롤링

[참고]  
- https://github.com/arkjackson/NaverMovieCommentCrawling
- https://github.com/JangBogwang/contentscrawling

## Watcha Pedia 크롤링 테스트

In [14]:
from tqdm.notebook import tqdm
from selenium.common.exceptions import NoSuchElementException

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
data = [{'title': '2001 스페이스 오디세이'},
        {'title': '티파니에서 아침을'},
        {'title': '미스 슬로운'},
        {'title': '휴일'}
        ]

title = pd.DataFrame(data)
title

,title
0,2001 스페이스 오디세이
1,티파니에서 아침을
2,미스 슬로운
3,휴일


In [4]:
# 검색할 영화 추출
inp = title['title']
inp

0    2001 스페이스 오디세이
1         티파니에서 아침을
2            미스 슬로운
3                휴일
Name: title, dtype: object

In [24]:
# 크롤링 
chrome_options = Options()
chrome_options.add_argument('--window-size=1800,1090')
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)
driver.get("https://pedia.watcha.com/ko-KR")
driver.implicitly_wait(4)

#크롤링 결과를 저장할 데이터 프레임 
result = pd.DataFrame(data=None, columns=["title",'img_url','year','genre','country', "star", 'smry',"review"])

for title in tqdm(inp):
    driver.get("https://pedia.watcha.com/ko-KR")
    driver.implicitly_wait(4)

    try:
        # 검색창에 영화 제목 입력
        search_box = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, 'desktop-search-field'))
        )
        search_box.send_keys(title)
        search_box.send_keys(Keys.RETURN)
        search_box.clear()
        driver.implicitly_wait(2)
        # b =[]
        # # 변수 선언 
        # img_url = ''
        # year = ''
        # genre = ''
        # point = ''
        # smry =''

        try:
            first_result = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul.css-1gkas1x-VisualUl li a"))
            )
            first_result.click()
            time.sleep(3)  # 페이지 로딩 대기
        except Exception as e:
            print(f"❌ '{title}' 검색 결과 없음: {e}")
            continue

    try:
        check = 0
        if driver.find_element(By.CSS_SELECTOR, 'div.css-1x6fzl1.e1szkzar0 > div.css-13o7eu2.e1pww8ij0 > section > section:nth-child(2) > header > h2').text!='영화':
            continue
        else:
            # 영화인지 체크 
            movie_list = driver.find_elements(By.CSS_SELECTOR, '#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section > div.css-1x6fzl1.e1szkzar0 > div.css-13o7eu2.e1pww8ij0 > section > section:nth-child(2) > div > div.css-a50hm3.e1yzo30x5 > div > ul > li')
            for i in range(1, len(movie_list)+1):
                movie = driver.find_element(By.CSS_SELECTOR,'#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section > div.css-1x6fzl1.e1szkzar0 > div.css-13o7eu2.e1pww8ij0 > section > section:nth-child(2) > div > div.css-a50hm3.e1yzo30x5 > div > ul > li:nth-child(%d) > a > div.css-1pkjqps.e1ic68ft0 > div.css-m5a6g.e1ic68ft1 > div.css-9jne0z.e1gexyxq4'%(i))
                if title== movie.text:
                    check = 1    
                    movie.click()
                    break;                        
        if check == 0:
            continue

        # 리뷰 url 저장  
        Url=driver.current_url+"/comments"
        
        #장르 연도 크롤링
        try:
            info_list = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[1]/div[4]/div/div[2]').text
            info_list = info_list.split(' · ')
            year = info_list[0]
            genre = info_list[1]
            country = info_list[2]
        except NoSuchElementException:
            pass
        
        #img url 저장 
        try:
            img = driver.find_element(By.XPATH,'//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section/div[1]/div/div/img')
            img_url = img.get_attribute('src')
        except NoSuchElementException:
            pass


        #줄거리 요약, 만약 줄거리가 없을 경우 빈 내용 저장 
        try:  
            smry = driver.find_element(By.CSS_SELECTOR, '#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > div > div.css-d3kywm.e11kyjwk3 > div > div.css-1oak90x.e11kyjwk1 > div > div.css-2tqido.e1yew2869 > section.css-3btvnx.e1yew2868 > div.css-g8juaf.e1yew2865 > section.css-291cts.e1yew2864').text
        except NoSuchElementException:
            pass

        # 별점 저장 
        try:
            point = driver.find_element(By.CSS_SELECTOR, '#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > div > div.css-d3kywm.e11kyjwk3 > div > div.css-1oak90x.e11kyjwk1 > div > div.css-2tqido.e1yew2869 > section > div.css-7ckxl2.e1yew2867 > section > span').text
            point = point.strip('평균 ★')
            point = point.split("(")
            star = point[0]
        except NoSuchElementException:
            pass

        driver.get(Url)
        #스크롤
        driver.execute_script("window.scrollTo(0, 2000)")
        time.sleep(0.3)
        driver.execute_script("window.scrollTo(2000, 4000)")
        time.sleep(0.3)
        driver.execute_script("window.scrollTo(4000, 6000)")
        time.sleep(0.3)
    
        # 리뷰 및 평점 크롤링 
        x = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[1]/section/section/div/div/div/ul/div')
        for i in range(0,len(x)-2) : 
            a = driver.find_element(By.CSS_SELECTOR,'#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section > div > div > div > ul > div:nth-child(%d) > div.css-2occzs.egj9y8a1'%(i+1)).text
            a = a.replace('\n','')
            a = a.replace('#','')
            b.append(a)

        # 데이터 추가 
        new = pd.DataFrame({"title":[title],'img_url':[img_url],'year':[year],'genre':[genre],'country':[country], "star":[star], 'smry':[smry],"review":[b]})
        result = pd.concat([result, new], ignore_index=True)
    except NoSuchElementException:
        pass

driver.close()

  0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
result

,title,img_url,year,genre,country,star,smry,review
